In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
from IPython.display import IFrame

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from generator import RoadNetwork
import osmnx as ox
import pandas as pd
import geopandas as gpd
import numpy as np
import fmm
from shapely.geometry import LineString
import swifter

In [2]:
network = RoadNetwork("Porto, Portugal", network_type="drive", retain_all=True, truncate_by_edge=True)

In [9]:
print(list(network.G.edges(data=True))[0:3])

[(25503936, 4722746638, {'osmid': 479127843, 'oneway': True, 'highway': 'motorway_link', 'length': 32.388}), (25503936, 281726624, {'osmid': 4256507, 'oneway': True, 'highway': 'motorway_link', 'length': 224.65500000000003, 'geometry': <shapely.geometry.linestring.LineString object at 0x133394550>}), (25503951, 1243857957, {'osmid': 4256513, 'oneway': True, 'lanes': '1', 'ref': 'IC 1', 'highway': 'motorway_link', 'maxspeed': '60', 'length': 115.10499999999999, 'geometry': <shapely.geometry.linestring.LineString object at 0x1333945e0>})]


In [10]:
network.save(path="../osm_data/porto")

/Users/ironmonkeyapps/Desktop/Studium/Master/Semester4/Abschlussarbeit/Road-Network-Embedding-Generator/generator/road_network.py:46: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_nodes.to_file(path + "/nodes.shp", encoding="utf-8")


In [7]:
td = pd.read_csv("../datasets/trajectories/Porto/mapped.csv")

In [22]:
tdtemp = td.loc[:, ["TRIP_ID", "POLYLINE"]]
tdtemp["id"] = np.arange(1, tdtemp.shape[0]+1)

In [23]:
tdtemp.to_csv("../datasets/trajectories/Porto/mapped_id_poly.csv", sep=";", index=False)

In [24]:
fnetwork = fmm.Network("../osm_data/porto/edges.shp", "fid","u","v")
graph = fmm.NetworkGraph(fnetwork)

#ubodt_gen = fmm.UBODTGenAlgorithm(fnetwork, graph)
#ubodt_gen.generate_ubodt("./ubodt.txt", 4, binary=False, use_omp=True)
ubodt = fmm.UBODT.read_ubodt_csv("./ubodt.txt")

model = fmm.FastMapMatch(fnetwork, graph, ubodt)

k = 8
radius = 0.003
gps_error = 0.0005
fmm_config = fmm.FastMapMatchConfig(k,radius,gps_error)

input_config = fmm.GPSConfig()
input_config.file = "../datasets/trajectories/Porto/mapped_id_poly.csv"
input_config.id = "id"
input_config.geom = "POLYLINE"
print(input_config.to_string())

result_config = fmm.ResultConfig()
result_config.file = "../datasets/trajectories/Porto/mr.txt"
result_config.output_config.write_opath = True
print(result_config.to_string())

[2022-05-04 19:48:16.441] [info] [network.cpp:72] Read network from file ../osm_data/porto/edges.shp
gps file : ../datasets/trajectories/Porto/mapped_id_poly.csv
id column : id
geom column : POLYLINE
timestamp column : timestamp
x column : x
y column : y
GPS point : false

Result file : ../datasets/trajectories/Porto/mr.txt
Output fields: opath cpath mgeom 
[2022-05-04 19:48:16.774] [info] [network.cpp:170] Number of edges 11308 nodes 5347
[2022-05-04 19:48:16.774] [info] [network.cpp:172] Field index: id 17 source 0 target 1
[2022-05-04 19:48:16.794] [info] [network.cpp:174] Read network done.
[2022-05-04 19:48:16.993] [info] [network_graph.cpp:17] Construct graph from network edges start
[2022-05-04 19:48:17.474] [info] [network_graph.cpp:30] Graph nodes 5347 edges 11308
[2022-05-04 19:48:17.475] [info] [network_graph.cpp:31] Construct graph from network edges end
[2022-05-04 19:48:17.567] [info] [ubodt.cpp:208] Reading UBODT file (CSV format) from ./ubodt.txt
[2022-05-04 19:48:18.44

In [25]:
status = model.match_gps_file(input_config, result_config, fmm_config, use_omp = True)
print(status)

In [5]:
G = network.G
gdf_nodes, gdf_edges = ox.graph_to_gdfs(
        G,
        nodes=True, edges=True,
        node_geometry=True,
        fill_edge_geometry=True)

gdf_edges.describe(include='all')

/Users/ironmonkeyapps/miniconda3/envs/road-emb/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,osmid,oneway,highway,length,geometry,lanes,ref,maxspeed,bridge,name,width,access,junction,tunnel,area
count,11308.0,11308,11308,11308.000000,11308,2316,480,854,164,10203,205,223,198,74,6
unique,4974.0,2,19,NaN,11304,12,32,18,4,1910,9,5,2,3,1
top,12301715.0,False,residential,NaN,"LINESTRING (-8.6667028 41.1481788, -8.6668598 ...",2,EN 12,50,yes,Avenida de Fernão de Magalhães,4,yes,roundabout,yes,highway
freq,30.0,7400,5779,NaN,2,1268,344,632,128,102,170,182,193,47,6
mean,NaN,NaN,NaN,85.117299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,94.203150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,1.564000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,30.470000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,60.193000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,107.726500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
m1 = ox.plot_graph_folium(G, popup_attribute="name", weight=2, color="#FFFFFF", tiles="cartodbdark_matter")
filepath = "graph.html"
m1.save(filepath)
#IFrame(filepath, width=600, height=500)

In [4]:
# load porto trajectory dataset
td = pd.read_csv("../datasets/trajectories/Porto/train.csv")

In [5]:
td = td[td["MISSING_DATA"]==False]
sample = np.matrix(td.loc[0, "POLYLINE"]).reshape(-1, 2)
gtd = gpd.GeoDataFrame(data=sample, index=np.arange(1, len(sample)+1), columns=["LONG", "LAT"], crs="EPSG:4326", geometry=gpd.points_from_xy(sample[:, 0], sample[:, 1]))
print(gtd.to_wkt())
#network.map_trajectorie(coordinates=gtd)

        LONG        LAT                     geometry
1  -8.618643  41.141412  POINT (-8.618643 41.141412)
2  -8.618499  41.141376  POINT (-8.618499 41.141376)
3  -8.620326  41.142510   POINT (-8.620326 41.14251)
4  -8.622153  41.143815  POINT (-8.622153 41.143815)
5  -8.623953  41.144373  POINT (-8.623953 41.144373)
6  -8.626680  41.144778   POINT (-8.62668 41.144778)
7  -8.627373  41.144697  POINT (-8.627373 41.144697)
8  -8.630226  41.145210   POINT (-8.630226 41.14521)
9  -8.632746  41.146920   POINT (-8.632746 41.14692)
10 -8.631738  41.148225  POINT (-8.631738 41.148225)
11 -8.629938  41.150385  POINT (-8.629938 41.150385)
12 -8.629110  41.151213   POINT (-8.62911 41.151213)
13 -8.629128  41.151240   POINT (-8.629128 41.15124)
14 -8.628786  41.152203  POINT (-8.628786 41.152203)
15 -8.628687  41.152374  POINT (-8.628687 41.152374)
16 -8.628759  41.152518  POINT (-8.628759 41.152518)
17 -8.630838  41.152680   POINT (-8.630838 41.15268)
18 -8.632323  41.153022  POINT (-8.632323 41.1